<a href="https://colab.research.google.com/github/satishchaudhary382/RFM/blob/master/wattar%20adar%20explosion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np 
import pandas_datareader as pdr
import matplotlib.pyplot as plt

In [ ]:
df = pdr.get_data_yahoo("COST","1/1/2018","1/1/2019")

In [ ]:
def Hanning(size):
    w = np.hanning(size+2)
    w = np.array(w[1:-1])  # remove zeros at endpoints
    return (w / max(w))

def wma(fs, size, pad=True, winType=Hanning, wts=None):
    """Apply a weighted moving average on the supplied series.
    Parameters
    ----------
    s : pandas.Series
        data to be averaged
    size : integer
        how wide a window to use
    pad : Boolean (optional, default = True)
        flag determining whether to pad beginning and end of data with a
        weighted average of the last `size` points. This provides better
        smoothing at the beginning and end of the line, but it tends to have
        zero slope.
    winType : Function (optional, default = Hanning)
        Window function that takes an integer (window size) and returns a list
        of weights to be applied to the data. The default is Hanning, a
        weighted cosine with non-zero endpoints. Other possible windows are:
        * np.bartlett (triangular with endpoints of 0)
        * np.blackman (3 cosines creating taper)
        * np.hamming (weighted cosine)
        * np.hanning (weighted cosine with endpoints of 0)
        * Triangle (triangle with non-zero enpoints, and option to
                    clip top of triangle)
    wts : list (optional, default = None)
        List of weights to use. `size` becomes the length of wts. Use this
        option to provide a custom weighting function. The length of wts
        should be odd, but this is not enforced.
    Returns
    -------
    Pandas Series containing smoothed data
    Notes
    -----
    Defaults to using a Hanning window for weights, centered on
    each point. For points near the beginning or end of data, special
    processing is required that isn't in built-in functions.
    Any rows with no value (nan) are dropped from series, and that reduced
    series is returned. This series will have fewer members than what was
    given, and may cause problems with mismatched indexes.
    """
    def SetLimits(i, hw):
        # i: current data location where window is centred
        # hw: half window width
        ds = max(0, (i-hw))         # data start
        de = min(n-1, (i+hw)) # data end
        ws = hw - (i - ds)          # window start
        we = hw + (de - i)          # window end
        return ds, de, ws, we

    s = fs.dropna()
    if type(wts) == type(None):
        size += (size+1) % 2  # make odd
        window = winType(size)
        window /= window.sum()  # normalize window
    else:
        window = wts / wts.sum()
        size = len(wts)
    n = len(s)
    hw = int(size / 2) # half window width
    # convolve has boundary effects when there is no overlap with the window
    # Begining and end of 'a' must be adjusted to compensate.
    # np.average() effectively scales the weights for the different sizes.
    if pad: # pad the data with reflected values
        # create padded beginning
        y = np.zeros(n+2*hw)
        for i in range(hw):
            y[i] = s.iloc[hw-i]
        for i in range(hw):
            y[i+n+hw] = s.iloc[n-i-1]
        for i in range(n):
            y[i+hw] = s.iloc[i]
        yc = np.convolve(y, window, mode='same')
        a = pd.Series(yc[hw:n+hw],
                      index=s.index,
                      name=s.name)
    else: # clip window as available data decreases
        a = pd.Series(np.convolve(s, window, mode='same'),
                      index=s.index,
                      name=s.name)
        for i in range(hw+1):  # fix the start
            (ds, de, ws, we) = SetLimits(i, hw)
            a.iloc[i] = np.average(s.iloc[ds:de], weights=window[ws:we])
        for i in range(n-hw-1, n):  # fix the end
            (ds, de, ws, we) = SetLimits(i, hw)
            a.iloc[i] = np.average(s.iloc[ds:de], weights=window[ws:we])
    return a

In [ ]:
def hma(df, n):
    a = 2 * wma(df, n/2)
    b = wma(df, n)
    return wma(a-b, np.sqrt(n))
# n = 25
# df['n/2'] = 2 * wma(df.Close,n/25)
# df['wma'] = wma(df.Close, n)
# df['hma'] = wma(df.wma, np.sqrt(n))
hma(df.Close,14).plot(figsize = (20,10))
df.Close.plot()

In [ ]:
# study("Waddah Attar Explosion [LazyBear]", shorttitle="WAE_LB")
# sensitivity = input(150, title="Sensitivity")
# fastLength=input(20, title="FastEMA Length")
# slowLength=input(40, title="SlowEMA Length")
# channelLength=input(20, title="BB Channel Length")
# mult=input(2.0, title="BB Stdev Multiplier")
# deadZone=input(20, title="No trade zone threshold")
sensitivity = 150
fast_length = 20
slow_length = 40
channel_length = 20
mult = 2
deadzone = 20

In [ ]:
# calc_macd(source, fastLength, slowLength) =>
# 	fastMA = ema(source, fastLength)
# 	slowMA = ema(source, slowLength)
# 	fastMA - slowMA

# calc_BBUpper(source, length, mult) => 
# 	basis = sma(source, length)
# 	dev = mult * stdev(source, length)
# 	basis + dev

# calc_BBLower(source, length, mult) => 
# 	basis = sma(source, length)
# 	dev = mult * stdev(source, length)
# 	basis - devc
def cal_macd(df, fast_length, slow_length):
    fastMA = hma(df.Close, fast_length)
    slowMA = hma(df.Close, slow_length)
    return fastMA - slowMA

def BB(df, length, mult):
    basis = df.Close.rolling(length).mean()
    std = df.Close.rolling(length).std()
    return basis + mult*std, basis - mult*std
    

In [ ]:
# t1 = (calc_macd(close, fastLength, slowLength) - calc_macd(close[1], fastLength, slowLength))*sensitivity
# t2 = (calc_macd(close[2], fastLength, slowLength) - calc_macd(close[3], fastLength, slowLength))*sensitivity

# e1 = (calc_BBUpper(close, channelLength, mult) - calc_BBLower(close, channelLength, mult))
# e2 = (calc_BBUpper(close[1], channelLength, mult) - calc_BBLower(close[1], channelLength, mult))

t1 = cal_macd(df, fast_length, slow_length) - cal_macd(df.shift(1), fast_length, slow_length)
t2 = cal_macd(df.shift(2), fast_length, slow_length) - cal_macd(df.shift(3),fast_length, slow_length)

e1 = BB(df,channel_length, mult)[0] - BB(df, channel_length, mult)[1]
e2 = BB(df.shift(1), channel_length, mult)[0] - BB(df.shift(1),channel_length, mult)[1]

df['t1'] = t1
df['t2'] = t2
df['e1'] = e1
df['e2'] = e2

In [ ]:
def trendup(df):
    if df.t1>=0:
        return df.t1
    else:
        return 0
def trendown(df):
    if df.t1 < 0:
        return (-1*df.t1)
    else:
        return 0
    
def Trend(df):
    if df.t1>0:
        return df.t1
    else:
        return (-1*df.t1)
    
df['Trend'] = df.apply(trend ,axis =1)
df['Up'] = df.apply(trendup, axis =1)
df['Down'] = df.apply(trendown,axis= 1)

In [ ]:

fig, ax = plt.subplots(1,figsize=(20,10), sharex = True)
ax.plot(df.Close)
ax.bar(df.index, df.Up*100, color = "green",alpha = 0.5, width = 3)
ax.bar(df.index, df.Down*100, color = "red",alpha = 0.5, width = 3)
# ax[1].plot(df.e1, color = "black")
ax.grid()